In [ ]:
# Transformer 모델 구축 - Transformer Sentiment Classifier 감정 분류 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터 로드 & 확인: 결측치 제거(None, "")
# - 2. 토크나이저 적용: Hugging Face DistilBertTokenizer 베이스 모델 사용
# - 3. 데이터셋 -> DataLoader 변환: DistilBertTokenizer 베이스 모델 토크나이저에서 DataLoader 로 바로 변환, Custom Dataset 필요 없음
# - 4. 모델정의 & GPU설정 & 전이학습 & 본체 동결
# - 전이 학습: DistilBertForSequenceClassification 베이스 모델(distilbert-base-uncased), num_labels=2 긍정/부정 2개 클래스
# - 본체 동결: model.distilbert.parameters()는 사전학습된 본체(embedding + transformer 블록)의 모든 파라미터를 의미,
# - 따라서 학습은 classifier 레이어(pre_classifier, classifier)만 진행된다
# - 5. 최적화 설정 & 학습 루프
# - 최적화 설정: autocast(속도 향상) GradScaler(안정적 학습) 적용

# DistilBert 구조 특징 
# - BERT 계열 모델은 Transformer의 Encoder 부분만 사용한다
# - 입력 문장을 임베딩 -> 여러층의 Transformer Encoder 블록 -> [CLS]토큰 벡터 추출 -> 분류기(Classifier)

# DistilBertForSequenceClassification 내부 흐름
# 입력(DataLoader 배치): 
# - input_ids:(batch_size,seq_len)->(32,128)
# - attention_mask: (32,128)
# - labels:(32)
# DistilBertForSequenceClassification(
#   (distilbert): DistilBertModel(
#     (embeddings): Embeddings(
#       (word_embeddings): Embedding(30522, 768, padding_idx=0) : (32,128) -> (32,128,768) (batch_size,seq_len,hidden_dim)
#       (position_embeddings): Embedding(512, 768) : (32,128) -> (32,128,768) (batch_size,seq_len,hidden_dim)
#       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True) : (32,128,768) (batch_size,seq_len,hidden_dim) 유지
#       (dropout): Dropout(p=0.1, inplace=False) : (32,128,768) (batch_size,seq_len,hidden_dim) 유지
#     )
#     (transformer): Transformer(
#       (layer): ModuleList(
#         (0-5): 6 x TransformerBlock( : (32,128,768) (batch_size,seq_len,hidden_dim) 유지
#           (attention): DistilBertSdpaAttention(
#             (dropout): Dropout(p=0.1, inplace=False)
#             (q_lin): Linear(in_features=768, out_features=768, bias=True)
#             (k_lin): Linear(in_features=768, out_features=768, bias=True)
#             (v_lin): Linear(in_features=768, out_features=768, bias=True)
#             (out_lin): Linear(in_features=768, out_features=768, bias=True)
#           )
#           (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True) : (32,128,768) (batch_size,seq_len,hidden_dim) 유지
#           (ffn): FFN( : (32,128,768) (batch_size,seq_len,hidden_dim)
#             (dropout): Dropout(p=0.1, inplace=False)
#             (lin1): Linear(in_features=768, out_features=3072, bias=True)
#             (lin2): Linear(in_features=3072, out_features=768, bias=True)
#             (activation): GELUActivation()
#           )
#           (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True) : (32,128,768) (batch_size,seq_len,hidden_dim) 유지
#         )
#       )
#     ) Transformer 출력(output_layer_norm 이후) (32,128,768) (batch_size,seq_len,hidden_dim) shape을 가지며
#       내부 forward()에서 hidden_states[:, 0, :] 선택([CSL] 벡터를 추출) -> (32,768) (batch_size,hidden_dim) 변경 한다
#   ) 
#   (pre_classifier): Linear(in_features=768, out_features=768, bias=True) : (32,768) -> (32,768) (batch_size,hidden_dim)
#   (classifier): Linear(in_features=768, out_features=2, bias=True) : (32,768)@(768,2) -> (32,2) 입력 벡터와 가중치 행렬의 내적 계산
#   (dropout): Dropout(p=0.2, inplace=False) : (32,768) (batch_size,hidden_dim) 유지

In [2]:
# 데이터 로드 & 확인
from datasets import load_dataset

# Hugging Face datasets 라이브러리 함수(CSV,JSON.. 데이터 로드)
dataset = load_dataset( # DatasetDict 형태로 반환
    'csv', # csv 포멧 지정
    data_files={ # train/test 데이터 각각 지정
        'train': "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
        'test': "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
    },
    delimiter='\t' # 구분자, 현재 데이터에서는 탭으로 구분
)

# 결측치 제거
all_clean_train = dataset["train"].filter(lambda x: x["document"] is not None and x["document"].strip() != "")
all_clean_test = dataset["test"].filter(lambda x: x["document"] is not None and x["document"].strip() != "")

# 데이터 축소
clean_train = all_clean_train.select(range(10000))
clean_test = all_clean_test.select(range(5000))

print(clean_train)
print(clean_test)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10000
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
})


In [3]:
# 토크나이저 적용
from transformers import DistilBertTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

def tokenizer_function(batch):
    return tokenizer(
        batch["document"],   # 문자열 리스트만 들어옴
        padding="max_length",
        truncation=True,
        max_length=128 # 시퀀스 길이, 연산량 절반 이상 감소
    )

tokenized_train = clean_train.map(
    tokenizer_function,
    batched=True,
    remove_columns=["document"]
)

tokenized_test = clean_test.map(
    tokenizer_function,
    batched=True,
    remove_columns=["document"]
)

print(tokenized_train[0])
print(tokenized_test[0])

{'id': 9976970, 'label': 0, 'input_ids': [101, 1463, 30006, 1457, 30008, 29996, 30019, 30025, 1012, 1012, 100, 100, 1459, 30011, 30020, 29997, 30011, 29994, 30019, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': 6270596, 'label': 1, 'input_ids': [101, 100, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
# 데이터셋 -> DataLoader 변환
import torch
from torch.utils.data import DataLoader

# collate_fn 함수 : Hugging Face Dataset에서 꺼낸 샘플은 파이썬 dict 형태, collate_fn() 각 샘플을 모아 PyTorch 텐서로 변환
def collate_fn(batch): # batch 샘플 리스트 : [{'input_ids':[...],'attention_mask':[...],'label':[...]}, {...}, ...]
    input_ids = torch.tensor([ item['input_ids'] for item in batch ]) # 토큰화된 문장
    attention_mask = torch.tensor([ item['attention_mask'] for item in batch ]) # 패딩 여부
    labels = torch.tensor([ item['label'] for item in batch ]) # 감성 분류 라벨(0=부정/1=긍정)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# train: 학습용 데이터, 배치크기 16, epoch 마다 데이터 순서 섞음
train_loader = DataLoader(tokenized_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
# valid: 검증용 데이터, 학습 데이터에서 10%를 검증용으로 분리 Hugging Face에서 제공하는 train_test_split() 메서드
valid_loader = DataLoader(tokenized_train.train_test_split(test_size=0.1)['test'], batch_size=32, shuffle=True, collate_fn=collate_fn)
# test: 테스트 데이터, 성능 최종 평가용
test_loader = DataLoader(tokenized_test, batch_size=32, shuffle=True, collate_fn=collate_fn)

# train_loader 데이터 확인
for batch in train_loader:
    print(batch.keys())
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['labels'].shape)
    break

dict_keys(['input_ids', 'attention_mask', 'labels'])
torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


In [6]:
# 모델 정의
from transformers import DistilBertForSequenceClassification
from torch.amp import autocast # 최신 API
from torch.amp import GradScaler

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DistilBertForSequenceClassification 베이스 모델(distilbert-base-uncased), num_labels=2 긍정/부정 2개 클래스
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# DistilBERT 본체 동결(Feature Extraction, Parameter Freezing, Weight Freezing)
# - model.distilbert.parameters()는 사전학습된 본체(embedding + transformer 블록)의 모든 파라미터를 의미 한다
# - requires_grad = False로 설정하면 역전파 시 이 파라미터들은 업데이트되지 않는다
# - 따라서 학습은 classifier 레이어(pre_classifier, classifier)만 진행된다
for param in model.distilbert.parameters():
    param.requires_grad = False

# 모델에 GPU 설정
model.to(device)

# 모델 확인
print(model)

# 최적화 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

# 반복횟수
num_epochs = 3

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# 학습 루프: autocast(속도 향상) 적용, GradScaler(안정적 학습) 적용
# autocast 적용: 연산을 FP16(half precision)과 FP32(full precision)중 적절히 선택해서 실행
# - 속도 향상: 대부분의 연산을 FP16으로 처리해 GPU 연산 속도를 높인다
# - 안정성 유지: 손실이 큰 연산(예시:소프트맥스,레이어정규화)은 FP32로 자동 변환해 정확도를 보장한다
# GradScaler 적용: FP16 학습에서는 작은 값이 underflow(0으로 사라짐)될 위험이 있다
# - 안정적 학습 보장: GradScaler는 손실(loss)를 크게 스케일링해서 역전파 시 그래디언트가 사라지지 않도록 한다
# - 이후 업데이트 단계에서 다시 원래 크기로 되돌려 안정적인 학습을 보장한다. 즉 FP16 학습에서 발생할 수 있는 수치 불안정 문제를 해결하는 역할
from tqdm import tqdm # 시각화(진행바)

# DistilBert 구조 특징 
# - BERT 계열 모델은 Transformer의 Encoder 부분만 사용한다
# - 입력 문장을 임베딩 -> 여러층의 Transformer Encoder 블록 -> [CLS]토큰 벡터 추출 -> 분류기(Classifier)
# DistilBertForSequenceClassification 내부 흐름
# 1. 입력: (batch_size,seq_len)
# 2. Encoder(DistilBERT 본체) 
# - Embedding: (batch_size,seq_len,hidden_dim) 
# - Transformer Block(6개): (batch_size,seq_len,hidden_dim)
# 3. [CLS] 토큰 추출: (batch_size,hidden_dim)
# 4. Classifier 레이어: (batch_size,num_labels)
# 5. 출력: logits, loss
for epoch in range(num_epochs):
    model.train() # 학습 모드 지정
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):        
        optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

        # 학습데이터 GPU 지정
        input_ids = batch['input_ids'].to(device) # [32, 128]
        attention_mask = batch['attention_mask'].to(device) # [32, 128]
        labels = batch['labels'].to(device) # [32]

        # 모델 forward(autocast 영역) 내부 shape 변환
        # - 임베딩 레이어: (32,128) -> (32,128,768) 
        # - Transformer 블록: (32,128,768) -> (32,128,768)
        # - CLS 토큰 추출: (32,768) 
        # - Classifier 레이어: (32,768) -> (32,2) 
        # - outputs.logits.shape: (32,2) 
        # - outputs.loss: scalar
        with autocast('cuda'):
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # 손실함수 적재
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1} | Tran Loss: {total_loss / len(train_loader):.4f}')

Epoch 1: 100%|██████████| 313/313 [00:40<00:00,  7.68it/s]


Epoch 1 | Tran Loss: 0.6838


Epoch 2: 100%|██████████| 313/313 [00:40<00:00,  7.69it/s]


Epoch 2 | Tran Loss: 0.6724


Epoch 3: 100%|██████████| 313/313 [00:40<00:00,  7.69it/s]

Epoch 3 | Tran Loss: 0.6678


In [ ]:
# # 학습 루프

# for epoch in range(num_epochs):
#     model.train() # 학습 모드 지정
#     total_loss = 0
#     for batch in train_loader:        
#         optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

#         # 학습데이터 GPU 지정
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)

#         # 모델 예측
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

#         # 손실함수
#         loss = outputs.loss

#         loss.backward() # 오차역전파 코드, 미분 연산
#         optimizer.step() # 오차역전파 코드, 미분 연산 후 가중치/바이어스 파라미터 업데이트

#         # 손실함수 적재
#         total_loss += loss.item()
    
#     print(f'Epoch {epoch + 1} | Tran Loss: {total_loss / len(train_loader):.4f}')